# Heston Almost Exact Simulation

In this notebook we adapt the [Heston Almost Exact Simulation](https://github.com/nburgessx/Papers/tree/main/HestonSimulation) by Nicholas Burgess, into a qablet MC Model, and use it to price

- Vanilla Options
- Forward Starting Options
- Accumulator Cliquets

In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
from qablet_contracts.eq.vanilla import Option
from qablet_contracts.timetable import py_to_ts
from src.qablet_utils import option_prices
from src.models.aes import HestonAESMC
from src.models.basic import HestonMCModel
from src.models.closed import price_vanilla_call

## Create Dataset
Create the dataset, with MC params, discounts and fwds as required by the [Dataset API](https://qablet-academy.github.io/intro/dataset/) 

For the `HESTON` section, add components as needed by the methods defined above. 

In [ ]:
times = np.array([0.0, 5.0])
rates = np.array([0.1, 0.1])
discount_data = ("ZERO_RATES", np.column_stack((times, rates)))

ticker = "EQ"
spot = 100.0
div_rate = 0.0
fwds = spot * np.exp((rates - div_rate) * times)
fwd_data = ("FORWARDS", np.column_stack((times, fwds)))
pricing_datetime = datetime(2023, 12, 31)

dataset = {
    "BASE": "USD",
    "PRICING_TS": py_to_ts(pricing_datetime).value,
    "ASSETS": {"USD": discount_data, ticker: fwd_data},
    "MC": {
        "PATHS": 2_500,
        "TIMESTEP": 1 / 1000,
        "SEED": 1,
    },
    "HESTON": {
        "ASSET": ticker,
        "INITIAL_VAR": 0.04,
        "LONG_VAR": 0.04,
        "VOL_OF_VAR": 1.0,
        "MEANREV": 0.5,
        "CORRELATION": -0.9,
    },
}

## Calculate Single Option Price

Define a [Vanilla Option](https://qablet.github.io/qablet-contracts/examples/equity_vanilla/) and calculate price.

In [ ]:
# Create Contract
strike = 100
ticker = "EQ"
timetable = Option(
    "USD", ticker, strike=strike, maturity=datetime(2024, 12, 31), is_call=True
).timetable()
print(timetable["events"].to_pandas())

  track                      time op  quantity unit
0       2024-12-31 00:00:00+00:00  >       0.0  USD
1       2024-12-31 00:00:00+00:00  +    -100.0  USD
2       2024-12-31 00:00:00+00:00  +       1.0   EQ


In [ ]:
model_aes = HestonAESMC()
price_aes, stats = model_aes.price(timetable, dataset)
print(f"price: {price_aes:11.6f}")

price:   12.383923


In [ ]:
model_basic = HestonMCModel()
price, stats = model_aes.price(timetable, dataset)
print(f"price: {price:11.6f}")

price:   12.431101


In [ ]:
# Closed form price
closed_price, _ = price_vanilla_call(
    strike,
    1.0,
    ticker,
    dataset,
)
print(f"closed_price: {closed_price:11.6f}")

closed_price:   12.331475


## Generate Multiple Option prices

In [ ]:
expirations = [
    datetime(2024, 3, 31),
    datetime(2024, 6, 30),
    datetime(2024, 12, 31),
]
strikes = np.array([0.8, 0.9, 1.0, 1.1, 1.2]) * spot
is_call = True

In [ ]:
aes_prices_df = option_prices(
    ticker, expirations, strikes, is_call, model_aes, dataset
)
print(aes_prices_df)

   Strike  2024-03-31  2024-06-30  2024-12-31
0    80.0   22.543674   24.841738   28.794924
1    90.0   13.372161   15.995866   20.388320
2   100.0    4.967488    7.599193   12.263109
3   110.0    0.169816    0.913060    4.764587
4   120.0    0.006676    0.051365    0.428029


In [ ]:
prices_df = option_prices(
    ticker, expirations, strikes, is_call, model_basic, dataset
)
print(prices_df)

   Strike  2024-03-31  2024-06-30  2024-12-31
0    80.0   22.263434   24.827605   29.123152
1    90.0   13.144785   15.948700   20.658092
2   100.0    4.853114    7.558823   12.451971
3   110.0    0.115870    0.874737    4.854035
4   120.0    0.001159    0.028506    0.427904


## Error vs Closed Form

In [ ]:
closed_price_df = pd.DataFrame.from_dict({"Strike": strikes})
for i, exp in enumerate(expirations):
    prc_ts = dataset["PRICING_TS"]
    # Get Time in years from the millisecond timestamps
    T = (py_to_ts(exp).value - prc_ts) / (365.25 * 24 * 3600 * 1e3)
    closed_prices = []
    for strike in strikes:
        closed_price, _ = price_vanilla_call(
            strike,
            T,
            ticker,
            dataset,
        )
        closed_prices.append(closed_price)

    exp_str = exp.strftime("%Y-%m-%d")
    closed_price_df[exp_str] = closed_prices

print(closed_price_df)

   Strike  2024-03-31  2024-06-30  2024-12-31
0    80.0   22.367712   24.774291   28.959679
1    90.0   13.245177   15.940640   20.524025
2   100.0    4.892852    7.544810   12.350271
3   110.0    0.127474    0.851502    4.776674
4   120.0    0.001781    0.039581    0.422797


In [ ]:
aes_prices_df - closed_price_df

,Strike,2024-03-31,2024-06-30,2024-12-31
0,0.0,-0.046007,0.140366,-0.137725
1,0.0,-0.027714,0.129245,-0.093447
2,0.0,-0.024372,0.113286,-0.031193
3,0.0,-0.008591,0.027709,0.019815
4,0.0,-0.000626,0.000250,-0.001555


In [ ]:
prices_df - closed_price_df

,Strike,2024-03-31,2024-06-30,2024-12-31
0,0.0,-0.104278,0.053314,0.163472
1,0.0,-0.100393,0.008060,0.134067
2,0.0,-0.039737,0.014014,0.101700
3,0.0,-0.011604,0.023235,0.077360
4,0.0,-0.000622,-0.011075,0.005107
